In [1]:
# import sys
# sys.path.insert(0, '..')

In [27]:
# from python_environment_check import check_packages


# d = {
#     'pandas': '1.3.2',
#     'torch': '1.9.0',
#     'torchtext': '0.11.0',
#     'datasets': '1.11.0',
#     'transformers': '4.9.1',
# }
# check_packages(d)

In [28]:
# import torch
# torch.cuda.is_available()

In [1]:
from IPython.display import Image

In [1]:
import gzip
import shutil
import time
import os

import pandas as pd
import numpy as np
import requests
import torch
import torch.nn.functional as F
#import torchtext
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

# import transformers
from transformers import DistilBertTokenizerFast, BertTokenizer, AutoTokenizer
from transformers import DistilBertForSequenceClassification, BertForSequenceClassification, AutoModel

In [2]:
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)
DEVICE = "cpu"
NUM_EPOCHS = 3

In [3]:
DEVICE

'cpu'

In [4]:
np.random.seed(123)
df = pd.read_csv('selection_main.csv', usecols=['abstract', 'class'])
df = df.sample(frac=1).reset_index(drop=True)
df.columns = ['review', 'sentiment']
df.head()

,review,sentiment
0,The existing volumetric gain for robotic explo...,Robotic
1,Combination therapies have become the standard...,Expert
2,Federated learning (FL) is a privacy-aware dat...,Cyber Security
3,"We propose V-Doc, a question-answering tool us...",Expert
4,HC4 is a new suite of test collections for ad ...,"Information Retrieval, Recommender Systems"


In [5]:
df['sentiment'].value_counts()

Robotic                                       1000
Expert                                        1000
Cyber Security                                1000
Information Retrieval, Recommender Systems    1000
Text Mining                                   1000
Control Systems                               1000
Fuzzy                                         1000
Computer Vision                               1000
Database                                      1000
Neural Nets                                   1000
Name: sentiment, dtype: int64

In [6]:
df['sentiment'].unique()

array(['Robotic', 'Expert', 'Cyber Security',
       'Information Retrieval, Recommender Systems', 'Text Mining',
       'Control Systems', 'Fuzzy', 'Computer Vision', 'Database',
       'Neural Nets'], dtype=object)

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
le.fit(df['sentiment'])

LabelEncoder()

In [10]:
le.transform(df['sentiment'])

array([8, 4, 2, ..., 0, 7, 5])

In [11]:
df['sentiment'].value_counts()

Robotic                                       1000
Expert                                        1000
Cyber Security                                1000
Information Retrieval, Recommender Systems    1000
Text Mining                                   1000
Control Systems                               1000
Fuzzy                                         1000
Computer Vision                               1000
Database                                      1000
Neural Nets                                   1000
Name: sentiment, dtype: int64

In [12]:
df['sentiment'] = le.transform(df['sentiment'])

In [13]:
df['sentiment'].value_counts()

8    1000
4    1000
2    1000
6    1000
9    1000
1    1000
5    1000
0    1000
3    1000
7    1000
Name: sentiment, dtype: int64

In [14]:
df['sentiment'].nunique()

10

In [15]:
df.head()

,review,sentiment
0,The existing volumetric gain for robotic explo...,8
1,Combination therapies have become the standard...,4
2,Federated learning (FL) is a privacy-aware dat...,2
3,"We propose V-Doc, a question-answering tool us...",4
4,HC4 is a new suite of test collections for ad ...,6


In [16]:
# text = df['review'].astype('str')
# labels = df['sentiment'].astype(int)
# train_texts, test_texts, train_labels, test_labels =\
# train_test_split(text, labels, train_size=0.5, stratify=labels)

train_texts = df.iloc[:7000]['review'].values
train_labels = df.iloc[:7000]['sentiment'].values

valid_texts = df.iloc[7000:8000]['review'].values
valid_labels = df.iloc[7000:8000]['sentiment'].values

test_texts = df.iloc[8000:]['review'].values
test_labels = df.iloc[8000:]['sentiment'].values

## RRRRRRRRRRR

In [16]:
df

,review,sentiment
0,The existing volumetric gain for robotic explo...,8
1,Combination therapies have become the standard...,4
2,Federated learning (FL) is a privacy-aware dat...,2
3,"We propose V-Doc, a question-answering tool us...",4
4,HC4 is a new suite of test collections for ad ...,6
...,...,...
9995,Stock market prediction has been an active are...,2
9996,"Closed-loop reservoir management (CLRM), in wh...",1
9997,While language tasks are naturally expressed i...,0
9998,We present the efficient implementations of pr...,7


In [17]:
df.review.values

array(['The existing volumetric gain for robotic exploration is calculated in the 3D occupancy map, while the sampling-based exploration method is extended in the reachable (free) space. The inconsistency between them makes the existing calculation of volumetric gain inappropriate for a complete exploration of the environment. To address this issue, we propose a concave-hull based volumetric gain in a sampling-based exploration framework. The concave hull is constructed based on the viewpoints generated by Rapidly-exploring Random Tree (RRT) and the nodes that fail to expand. All space outside this concave hull is considered unknown. The volumetric gain is calculated based on the viewpoints configuration rather than using the occupancy map. With the new volumetric gain, robots can avoid inefficient or even erroneous exploration behavior caused by the inappropriateness of existing volumetric gain calculation methods. Our exploration method is evaluated against the existing state-of-the-

In [18]:
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab

from collections import Counter

from IPython.core.display import HTML, display

C:\Users\777\AppData\Local\Temp\ipykernel_26328\1131734440.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [19]:
list_df = []
for index, row in df.iterrows():
    x = list()
    x.append(row['sentiment'])
    x.append(row['review'])
    x = tuple(x)
    list_df.append(x)

In [20]:
len(list_df)

10000

In [21]:
list_df[0]

(8,
 'The existing volumetric gain for robotic exploration is calculated in the 3D occupancy map, while the sampling-based exploration method is extended in the reachable (free) space. The inconsistency between them makes the existing calculation of volumetric gain inappropriate for a complete exploration of the environment. To address this issue, we propose a concave-hull based volumetric gain in a sampling-based exploration framework. The concave hull is constructed based on the viewpoints generated by Rapidly-exploring Random Tree (RRT) and the nodes that fail to expand. All space outside this concave hull is considered unknown. The volumetric gain is calculated based on the viewpoints configuration rather than using the occupancy map. With the new volumetric gain, robots can avoid inefficient or even erroneous exploration behavior caused by the inappropriateness of existing volumetric gain calculation methods. Our exploration method is evaluated against the existing state-of-the-ar

In [22]:
ag_train, ag_val = list_df[:7000], list_df[7000:]

tokenizer = get_tokenizer('basic_english')
word_counter = Counter()
for (label, line) in list_df:
    word_counter.update(tokenizer(line))
voc = Vocab(word_counter)

print('Vocabulary size:', len(voc))

num_class = len(set(label for label, _ in ag_train))
print('Num of classes:', num_class)

Vocabulary size: 51897
Num of classes: 10


In [23]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")
train_iter = list_df


def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [24]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [25]:
from torch.utils.data import DataLoader

device = "cpu"


def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


train_iter = list_df
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

In [26]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [27]:
train_iter = list_df
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [28]:
import time


def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [29]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10  # epoch
LR = 5  # learning rate
BATCH_SIZE = 64  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = list_df[:7000], list_df[7000:]
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

IndexError: Target -1 is out of bounds.

(8,
 'The existing volumetric gain for robotic exploration is calculated in the 3D occupancy map, while the sampling-based exploration method is extended in the reachable (free) space. The inconsistency between them makes the existing calculation of volumetric gain inappropriate for a complete exploration of the environment. To address this issue, we propose a concave-hull based volumetric gain in a sampling-based exploration framework. The concave hull is constructed based on the viewpoints generated by Rapidly-exploring Random Tree (RRT) and the nodes that fail to expand. All space outside this concave hull is considered unknown. The volumetric gain is calculated based on the viewpoints configuration rather than using the occupancy map. With the new volumetric gain, robots can avoid inefficient or even erroneous exploration behavior caused by the inappropriateness of existing volumetric gain calculation methods. Our exploration method is evaluated against the existing state-of-the-art RRT-based method in a benchmark environment. In the evaluated environment, the average running time of our method is about 38.4% of the existing state-of-the-art method and our method is more robust.')

In [ ]:
test_label = 8  # {1: World, 2: Sports, 3: Business, 4: Sci/Tec}
test_line = ('The existing volumetric gain for robotic exploration is calculated in the 3D occupancy map, while the sampling-based exploration method is extended in the reachable (free) space. The inconsistency between them makes the existing calculation of volumetric gain inappropriate for a complete exploration of the environment. To address this issue, we propose a concave-hull based volumetric gain in a sampling-based exploration framework. The concave hull is constructed based on the viewpoints generated by Rapidly-exploring Random Tree (RRT) and the nodes that fail to expand. All space outside this concave hull is considered unknown. The volumetric gain is calculated based on the viewpoints configuration rather than using the occupancy map. With the new volumetric gain, robots can avoid inefficient or even erroneous exploration behavior caused by the inappropriateness of existing volumetric gain calculation methods. Our exploration method is evaluated against the existing state-of-the-art RRT-based method in a benchmark environment. In the evaluated environment, the average running time of our method is about 38.4% of the existing state-of-the-art method and our method is more robust.')

test_labels, test_text, test_offsets = collate_batch([(test_label, test_line)])

probs = F.softmax(model(test_text, test_offsets), dim=1).squeeze(0)
print('Prediction probability:', round(probs[test_labels[0]].item(), 4))

In [66]:
class EmbeddingBagModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        self.linear = nn.Linear(embed_dim, num_class)

    def forward(self, inputs, offsets):
        embedded = self.embedding(inputs, offsets)
        return self.linear(embedded)

In [67]:
BATCH_SIZE = 64

def collate_batch(batch):
    labels = torch.tensor([label - 1 for label, _ in batch])
    text_list = [tokenizer(line) for _, line in batch]

    # flatten tokens across the whole batch
    text = torch.tensor([voc[t] for tokens in text_list for t in tokens])
    # the offset of each example
    offsets = torch.tensor(
        [0] + [len(tokens) for tokens in text_list][:-1]
    ).cumsum(dim=0)

    return labels, text, offsets

train_loader = DataLoader(ag_train, batch_size=BATCH_SIZE,
                          shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(ag_val, batch_size=BATCH_SIZE,
                        shuffle=False, collate_fn=collate_batch)

In [54]:
EPOCHS = 7
EMB_SIZE = 64
CHECKPOINT = './models/embedding_bag_ag_news.pt'
USE_PRETRAINED = True  # change to False if you want to retrain your own model

def train_model(train_loader, val_loader):
    model = EmbeddingBagModel(len(voc), EMB_SIZE, num_class)

    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())

    for epoch in range(1, EPOCHS + 1):
        # training
        model.train()
        total_acc, total_count = 0, 0

        for idx, (label, text, offsets) in enumerate(train_loader):
            optimizer.zero_grad()
            predited_label = model(text, offsets)
            loss(predited_label, label).backward()
            optimizer.step()
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

            if (idx + 1) % 500 == 0:
                print('epoch {:3d} | {:5d}/{:5d} batches | accuracy {:8.3f}'.format(
                    epoch, idx + 1, len(train_loader), total_acc / total_count
                ))
                total_acc, total_count = 0, 0

        # evaluation
        model.eval()
        total_acc, total_count = 0, 0

        with torch.no_grad():
            for label, text, offsets in val_loader:
                predited_label = model(text, offsets)
                total_acc += (predited_label.argmax(1) == label).sum().item()
                total_count += label.size(0)

        print('-' * 59)
        print('end of epoch {:3d} | valid accuracy {:8.3f} '.format(epoch, total_acc / total_count))
        print('-' * 59)

    torch.save(model, CHECKPOINT)
    return model

eb_model = torch.load(CHECKPOINT) if USE_PRETRAINED else train_model(train_loader, val_loader)

RuntimeError: Expected i < index_size to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

## Tokenizing the dataset

In [17]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [18]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
#                                           do_lower_case=True)

In [19]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

In [20]:
train_encodings[0]

Encoding(num_tokens=476, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

**Dataset Class and Loaders**

In [21]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(valid_encodings, valid_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [22]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

## Loading and fine-tuning a pre-trained BERT model

In [23]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                            num_labels=df['sentiment'].nunique(),
                                                            output_attentions=False,
                                                            output_hidden_states=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.

In [24]:
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=df['sentiment'].nunique(),
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)

In [25]:
model.to('cuda')
model.train()

optim = torch.optim.Adam(model.parameters(), lr=5e-5)

**Train Model -- Manual Training Loop**

In [26]:
DEVICE

device(type='cuda')

In [27]:
def compute_accuracy(model, data_loader, device):
    with torch.no_grad():
        correct_pred, num_examples = 0, 0
        
        for batch_idx, batch in enumerate(data_loader):
        
        ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs['logits']
            predicted_labels = torch.argmax(logits, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()
        
        return correct_pred.float()/num_examples * 100


In [28]:
%%time
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, batch in enumerate(train_loader):
        
        ### Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        batch['labels'] = batch['labels'].type(torch.LongTensor)
        labels = batch['labels'].to(DEVICE)

        ### Forward
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
        
        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
                   f'Loss: {loss:.4f}')
            
    model.eval()

    with torch.set_grad_enabled(False):
        print(f'Training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nValid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
# print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 0001/0003 | Batch 0000/0438 | Loss: 2.2791
Epoch: 0001/0003 | Batch 0250/0438 | Loss: 0.1799
Training accuracy: 92.01%
Valid accuracy: 92.10%
Time elapsed: 1.02 min
Epoch: 0002/0003 | Batch 0000/0438 | Loss: 0.8439
Epoch: 0002/0003 | Batch 0250/0438 | Loss: 0.0351
Training accuracy: 97.41%
Valid accuracy: 95.20%
Time elapsed: 1.99 min
Epoch: 0003/0003 | Batch 0000/0438 | Loss: 0.4293
Epoch: 0003/0003 | Batch 0250/0438 | Loss: 0.0106
Training accuracy: 98.13%
Valid accuracy: 96.20%
Time elapsed: 2.95 min
Total Training Time: 2.95 min
CPU times: total: 2min 32s
Wall time: 2min 57s


In [29]:
import torch
import torch.nn.functional as F

from captum.attr import visualization as viz
from captum.attr import Lime, LimeBase
from captum._utils.models.linear_model import SkLearnLinearRegression, SkLearnLasso

import os
import json

In [30]:
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab

from collections import Counter

from IPython.core.display import HTML, display

C:\Users\777\AppData\Local\Temp\ipykernel_7688\1131734440.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [31]:
# remove the batch dimension for the embedding-bag model
def forward_func(text, offsets):
    return model(text.squeeze(0), offsets)

# encode text indices into latent representations & calculate cosine similarity
def exp_embedding_cosine_distance(original_inp, perturbed_inp, _, **kwargs):
    original_emb = model.embedding(original_inp, None)
    perturbed_emb = model.embedding(perturbed_inp, None)
    distance = 1 - F.cosine_similarity(original_emb, perturbed_emb, dim=1)
    return torch.exp(-1 * (distance ** 2) / 2)

# binary vector where each word is selected independently and uniformly at random
def bernoulli_perturb(text, **kwargs):
    probs = torch.ones_like(text) * 0.5
    return torch.bernoulli(probs).long()

# remove absenst token based on the intepretable representation sample
def interp_to_input(interp_sample, original_input, **kwargs):
    return original_input[interp_sample.bool()].view(original_input.size(0), -1)

lasso_lime_base = LimeBase(
    forward_func,
    interpretable_model=SkLearnLasso(alpha=0.08),
    similarity_func=exp_embedding_cosine_distance,
    perturb_func=bernoulli_perturb,
    perturb_interpretable_space=True,
    from_interp_rep_transform=interp_to_input,
    to_interp_rep_transform=None
)

In [36]:
df.review[0]

'The existing volumetric gain for robotic exploration is calculated in the 3D occupancy map, while the sampling-based exploration method is extended in the reachable (free) space. The inconsistency between them makes the existing calculation of volumetric gain inappropriate for a complete exploration of the environment. To address this issue, we propose a concave-hull based volumetric gain in a sampling-based exploration framework. The concave hull is constructed based on the viewpoints generated by Rapidly-exploring Random Tree (RRT) and the nodes that fail to expand. All space outside this concave hull is considered unknown. The volumetric gain is calculated based on the viewpoints configuration rather than using the occupancy map. With the new volumetric gain, robots can avoid inefficient or even erroneous exploration behavior caused by the inappropriateness of existing volumetric gain calculation methods. Our exploration method is evaluated against the existing state-of-the-art RRT

In [44]:
tokenizer = get_tokenizer('basic_english')
word_counter = Counter()
for label, line in df.iterrows():
    word_counter.update(tokenizer(str(line)))
voc = Vocab(word_counter)

print('Vocabulary size:', len(voc))

num_class = len(set(label for label, _ in df.iterrows()))
print('Num of classes:', num_class)

Vocabulary size: 19828
Num of classes: 10000


In [45]:
def collate_batch(batch):
    labels = torch.tensor([label - 1 for label, _ in batch])
    text_list = [tokenizer(line) for _, line in batch]

    # flatten tokens across the whole batch
    text = torch.tensor([voc[t] for tokens in text_list for t in tokens])
    # the offset of each example
    offsets = torch.tensor(
        [0] + [len(tokens) for tokens in text_list][:-1]
    ).cumsum(dim=0)

    return labels, text, offsets

train_loader = DataLoader(train_dataset, batch_size=16,
                          shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(valid_dataset, batch_size=16,
                        shuffle=False, collate_fn=collate_batch)

In [50]:
test_label = 8  # {1: World, 2: Sports, 3: Business, 4: Sci/Tec}
test_line = ('The existing volumetric gain for robotic exploration is calculated in the 3D occupancy map, while the sampling-based exploration method is extended in the reachable (free) space. The inconsistency between them makes the existing calculation of volumetric gain inappropriate for a complete exploration of the environment. To address this issue, we propose a concave-hull based volumetric gain in a sampling-based exploration framework. The concave hull is constructed based on the viewpoints generated by Rapidly-exploring Random Tree (RRT) and the nodes that fail to expand. All space outside this concave hull is considered unknown. The volumetric gain is calculated based on the viewpoints configuration rather than using the occupancy map. With the new volumetric gain, robots can avoid inefficient or even erroneous exploration behavior caused by the inappropriateness of existing volumetric gain calculation methods. Our exploration method is evaluated against the existing state-of-the-art RRT-based method in a benchmark environment. In the evaluated environment, the average running time of our method is about 38.4% of the existing state-of-the-art method and our method is more robust.')

test_labels, test_text, test_offsets = collate_batch([(test_label, test_line)])

probs = F.softmax(model(test_text.to(DEVICE), test_offsets), dim=1).squeeze(0)
print('Prediction probability:', round(probs[test_labels[0]].item(), 4))

RuntimeError: The size of tensor a (191) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
attrs = lasso_lime_base.attribute(
    test_text.unsqueeze(0), # add batch dimension for Captum
    target=test_labels,
    additional_forward_args=(test_offsets,),
    n_samples=32000,
    show_progress=True
).squeeze(0)

print('Attribution range:', attrs.min().item(), 'to', attrs.max().item())

In [ ]:
def show_text_attr(attrs):
    rgb = lambda x: '255,0,0' if x < 0 else '0,255,0'
    alpha = lambda x: abs(x) ** 0.5
    token_marks = [
        f'<mark style="background-color:rgba({rgb(attr)},{alpha(attr)})">{token}</mark>'
        for token, attr in zip(tokenizer(test_line), attrs.tolist())
    ]

    display(HTML('<p>' + ' '.join(token_marks) + '</p>'))

show_text_attr(attrs)

In [29]:
# %%time
# print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

In [30]:
%%time
true_labels, pred_labels = [], []
with torch.no_grad():
    
    correct_pred, num_examples = 0, 0

    for batch_idx, batch in enumerate(test_loader):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs['logits']
        predicted_labels = torch.argmax(logits, 1)
        true_labels.append(labels.cpu().data.numpy())
        pred_labels.append(predicted_labels.cpu().data.numpy())

true_labels = np.array(true_labels).reshape(1, -1)[0]
pred_labels = np.array(pred_labels).reshape(1, -1)[0]

CPU times: total: 4.44 s
Wall time: 4.49 s


In [31]:
print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       183
           1       0.95      0.94      0.94       220
           2       0.98      0.95      0.97       191
           3       0.98      0.96      0.97       200
           4       0.92      0.97      0.94       190
           5       0.98      0.97      0.98       202
           6       0.97      0.91      0.94       215
           7       0.95      0.98      0.97       200
           8       0.95      0.99      0.97       203
           9       0.96      0.97      0.96       196

    accuracy                           0.96      2000
   macro avg       0.96      0.96      0.96      2000
weighted avg       0.96      0.96      0.96      2000



In [32]:
print(classification_report(le.inverse_transform(true_labels), le.inverse_transform(pred_labels)))

                                            precision    recall  f1-score   support

                           Computer Vision       0.99      0.97      0.98       183
                           Control Systems       0.95      0.94      0.94       220
                            Cyber Security       0.98      0.95      0.97       191
                                  Database       0.98      0.96      0.97       200
                                    Expert       0.92      0.97      0.94       190
                                     Fuzzy       0.98      0.97      0.98       202
Information Retrieval, Recommender Systems       0.97      0.91      0.94       215
                               Neural Nets       0.95      0.98      0.97       200
                                   Robotic       0.95      0.99      0.97       203
                               Text Mining       0.96      0.97      0.96       196

                                  accuracy                           0.96 

In [33]:
print(classification_report(true_labels, pred_labels, target_names=le.classes_))

                                            precision    recall  f1-score   support

                           Computer Vision       0.99      0.97      0.98       183
                           Control Systems       0.95      0.94      0.94       220
                            Cyber Security       0.98      0.95      0.97       191
                                  Database       0.98      0.96      0.97       200
                                    Expert       0.92      0.97      0.94       190
                                     Fuzzy       0.98      0.97      0.98       202
Information Retrieval, Recommender Systems       0.97      0.91      0.94       215
                               Neural Nets       0.95      0.98      0.97       200
                                   Robotic       0.95      0.99      0.97       203
                               Text Mining       0.96      0.97      0.96       196

                                  accuracy                           0.96 

In [34]:
from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients